In [19]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

### 데이터를 불러옴

In [20]:
data = pd.read_csv("칼로리 정보.csv", engine='python', encoding='CP949')
data

,음식명,1인분칼로리(kcal),탄수화물(g),단백질(g),지방(g),콜레스트롤(g),식이섬유(g),나트륨(g),등록일
0,가래떡(떡국용),311.0,68.3,5.3,1.0,0.0,0.0,231.0,NaN
1,가래떡(떡볶이용),96.0,21.0,1.6,0.3,0.0,0.0,71.0,NaN
2,가자미,58.0,0.1,9.9,1.7,44.6,0.0,104.0,NaN
3,가자미구이,76.0,0.2,11.2,3.1,50.1,0.0,255.0,2012-01-06
4,가자미조림,45.4,2.4,6.6,1.2,27.0,0.4,183.0,2012-01-04
...,...,...,...,...,...,...,...,...,...
632,효모,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2012-01-04
633,후렌치후라이,367.0,47.2,4.4,19.7,0.0,4.0,223.0,2012-01-04
634,후르츠샐러드(칵테일),29.0,7.7,0.1,0.2,0.0,0.2,1.0,2012-01-04
635,후추,3.0,0.7,0.1,0.0,0.0,0.1,0.0,NaN


### 결측치 유무 및 데이터 정보확인

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   음식명           637 non-null    object 
 1   1인분칼로리(kcal)  637 non-null    float64
 2   탄수화물(g)       637 non-null    float64
 3   단백질(g)        637 non-null    float64
 4   지방(g)         637 non-null    float64
 5   콜레스트롤(g)      637 non-null    float64
 6   식이섬유(g)       637 non-null    float64
 7   나트륨(g)        637 non-null    float64
 8   등록일           474 non-null    object 
dtypes: float64(7), object(2)
memory usage: 44.9+ KB


In [22]:
data.isnull().sum()

음식명               0
1인분칼로리(kcal)      0
탄수화물(g)           0
단백질(g)            0
지방(g)             0
콜레스트롤(g)          0
식이섬유(g)           0
나트륨(g)            0
등록일             163
dtype: int64

In [23]:
data.columns

Index(['음식명', '1인분칼로리(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '콜레스트롤(g)',
       '식이섬유(g)', '나트륨(g)', '등록일'],
      dtype='object')

### 새로운 column 생성(g → % 변경)

In [24]:
total_gram = (data['탄수화물(g)']+ data['지방(g)'] + data['단백질(g)'])

data['탄수화물(%)'] = data['탄수화물(g)'] / total_gram * 100
data['단백질(%)'] = data['단백질(g)'] / total_gram * 100
data['지방(%)'] = data['지방(g)'] / total_gram * 100
# data['나트륨(%)'] = data['나트륨(g)'] / total_gram * 100

### 결측치 제거(모든 데이터가 0의 값을 띄기때문에 제거함 ex) 효모 : 칼로리/탄/단/지 : 0 )

In [25]:
print(data.shape)
data_n = data[data['단백질(%)'].notna()]
print(data_n.shape)

(637, 12)
(631, 12)


In [26]:
data_n.head()

,음식명,1인분칼로리(kcal),탄수화물(g),단백질(g),지방(g),콜레스트롤(g),식이섬유(g),나트륨(g),등록일,탄수화물(%),단백질(%),지방(%)
0,가래떡(떡국용),311.0,68.3,5.3,1.0,0.0,0.0,231.0,NaN,91.554960,7.104558,1.340483
1,가래떡(떡볶이용),96.0,21.0,1.6,0.3,0.0,0.0,71.0,NaN,91.703057,6.986900,1.310044
2,가자미,58.0,0.1,9.9,1.7,44.6,0.0,104.0,NaN,0.854701,84.615385,14.529915
3,가자미구이,76.0,0.2,11.2,3.1,50.1,0.0,255.0,2012-01-06,1.379310,77.241379,21.379310
4,가자미조림,45.4,2.4,6.6,1.2,27.0,0.4,183.0,2012-01-04,23.529412,64.705882,11.764706


### 필요없는 column 제거

In [27]:
del data_n['등록일']
data_n

,음식명,1인분칼로리(kcal),탄수화물(g),단백질(g),지방(g),콜레스트롤(g),식이섬유(g),나트륨(g),탄수화물(%),단백질(%),지방(%)
0,가래떡(떡국용),311.0,68.3,5.3,1.0,0.0,0.0,231.0,91.554960,7.104558,1.340483
1,가래떡(떡볶이용),96.0,21.0,1.6,0.3,0.0,0.0,71.0,91.703057,6.986900,1.310044
2,가자미,58.0,0.1,9.9,1.7,44.6,0.0,104.0,0.854701,84.615385,14.529915
3,가자미구이,76.0,0.2,11.2,3.1,50.1,0.0,255.0,1.379310,77.241379,21.379310
4,가자미조림,45.4,2.4,6.6,1.2,27.0,0.4,183.0,23.529412,64.705882,11.764706
...,...,...,...,...,...,...,...,...,...,...,...
631,회덮밥,696.5,121.9,39.8,5.5,51.2,2.5,892.0,72.906699,23.803828,3.289474
633,후렌치후라이,367.0,47.2,4.4,19.7,0.0,4.0,223.0,66.199158,6.171108,27.629734
634,후르츠샐러드(칵테일),29.0,7.7,0.1,0.2,0.0,0.2,1.0,96.250000,1.250000,2.500000
635,후추,3.0,0.7,0.1,0.0,0.0,0.1,0.0,87.500000,12.500000,0.000000


### 결측치 및 column 제거 확인

In [28]:
data_n.isnull().sum()

음식명             0
1인분칼로리(kcal)    0
탄수화물(g)         0
단백질(g)          0
지방(g)           0
콜레스트롤(g)        0
식이섬유(g)         0
나트륨(g)          0
탄수화물(%)         0
단백질(%)          0
지방(%)           0
dtype: int64

In [29]:
data_n.head()

,음식명,1인분칼로리(kcal),탄수화물(g),단백질(g),지방(g),콜레스트롤(g),식이섬유(g),나트륨(g),탄수화물(%),단백질(%),지방(%)
0,가래떡(떡국용),311.0,68.3,5.3,1.0,0.0,0.0,231.0,91.554960,7.104558,1.340483
1,가래떡(떡볶이용),96.0,21.0,1.6,0.3,0.0,0.0,71.0,91.703057,6.986900,1.310044
2,가자미,58.0,0.1,9.9,1.7,44.6,0.0,104.0,0.854701,84.615385,14.529915
3,가자미구이,76.0,0.2,11.2,3.1,50.1,0.0,255.0,1.379310,77.241379,21.379310
4,가자미조림,45.4,2.4,6.6,1.2,27.0,0.4,183.0,23.529412,64.705882,11.764706


### 라벨인코딩
   음식명(범주형)을 숫자로 변경하여 정확한 예측하게 함 

In [30]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [31]:
en_x = LabelEncoder()
data_n['음식명_lbl'] = en_x.fit_transform(data_n['음식명'])
data_n

<ipython-input-31-88a217c5b73e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_n['음식명_lbl'] = en_x.fit_transform(data_n['음식명'])


,음식명,1인분칼로리(kcal),탄수화물(g),단백질(g),지방(g),콜레스트롤(g),식이섬유(g),나트륨(g),탄수화물(%),단백질(%),지방(%),음식명_lbl
0,가래떡(떡국용),311.0,68.3,5.3,1.0,0.0,0.0,231.0,91.554960,7.104558,1.340483,0
1,가래떡(떡볶이용),96.0,21.0,1.6,0.3,0.0,0.0,71.0,91.703057,6.986900,1.310044,1
2,가자미,58.0,0.1,9.9,1.7,44.6,0.0,104.0,0.854701,84.615385,14.529915,2
3,가자미구이,76.0,0.2,11.2,3.1,50.1,0.0,255.0,1.379310,77.241379,21.379310,3
4,가자미조림,45.4,2.4,6.6,1.2,27.0,0.4,183.0,23.529412,64.705882,11.764706,4
...,...,...,...,...,...,...,...,...,...,...,...,...
631,회덮밥,696.5,121.9,39.8,5.5,51.2,2.5,892.0,72.906699,23.803828,3.289474,618
633,후렌치후라이,367.0,47.2,4.4,19.7,0.0,4.0,223.0,66.199158,6.171108,27.629734,619
634,후르츠샐러드(칵테일),29.0,7.7,0.1,0.2,0.0,0.2,1.0,96.250000,1.250000,2.500000,620
635,후추,3.0,0.7,0.1,0.0,0.0,0.1,0.0,87.500000,12.500000,0.000000,621


### 칼로리/탄수화물/단백질/지방 데이터 타입을 정수형으로 변경

In [14]:
data_n['1인분칼로리(kcal)'] = data_n['1인분칼로리(kcal)'].astype(int)
data_n['탄수화물(%)'] = data_n['탄수화물(%)'].astype(int)
data_n['단백질(%)'] = data_n['단백질(%)'].astype(int)
data_n['지방(%)'] = data_n['지방(%)'].astype(int)
data_n.head()

<ipython-input-14-9041f3787805>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_n['1인분칼로리(kcal)'] = data_n['1인분칼로리(kcal)'].astype(int)
<ipython-input-14-9041f3787805>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_n['탄수화물(%)'] = data_n['탄수화물(%)'].astype(int)
<ipython-input-14-9041f3787805>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

,음식명,1인분칼로리(kcal),탄수화물(g),단백질(g),지방(g),콜레스트롤(g),식이섬유(g),나트륨(g),탄수화물(%),단백질(%),지방(%),음식명_lbl
0,가래떡(떡국용),311,68.3,5.3,1.0,0.0,0.0,231.0,91,7,1,0
1,가래떡(떡볶이용),96,21.0,1.6,0.3,0.0,0.0,71.0,91,6,1,1
2,가자미,58,0.1,9.9,1.7,44.6,0.0,104.0,0,84,14,2
3,가자미구이,76,0.2,11.2,3.1,50.1,0.0,255.0,1,77,21,3
4,가자미조림,45,2.4,6.6,1.2,27.0,0.4,183.0,23,64,11,4


### 데이터를 나눔

In [136]:
sel = ['1인분칼로리(kcal)','탄수화물(%)','단백질(%)','지방(%)']
X = data_n[sel]
y = data_n['음식명_lbl']

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.1,
                                                   random_state = 0)

print(X_train.shape)

(567, 4)


### knn모델 학습

In [137]:
### 
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
model = KNeighborsClassifier()
model.fit(X_train, y_train)


KNeighborsClassifier()

###  사용자로부터 입력 받음

In [138]:

print("4가지 정보를 입력해 주세요! 구분은 공백으로 (a,b,c,d)")
print("a : 칼로리(kcal), b : 탄수화물(%), c : 단백질(%), d : 지방(%) ")
a, b, c, d = input().split()
inp_value = [a,b,c,d]

4가지 정보를 입력해 주세요! 구분은 공백으로 (a,b,c,d)
a : 칼로리(kcal), b : 탄수화물(%), c : 단백질(%), d : 지방(%) 
300 90 5 5


In [139]:
pred = model.predict([inp_value])
# print(pred, type(pred))
pred[0]

C:\Users\hingu\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


0

### loc를 이용하여 음식명 추출

In [140]:
sel_food = data_n.loc[ data_n['음식명_lbl']==pred[0], "음식명"]
print("추천 음식은 ", sel_food.values)

추천 음식은  ['가래떡(떡국용)']


### knn 모델 정확도

In [141]:
print("학습용 데이터 세트 점수 : {:.2f}".format(model.score(X_train, y_train)))
print("테스트 데이터 세트 점수 : {:.2f}".format(model.score(X_test, y_test)))

학습용 데이터 세트 점수 : 0.21
테스트 데이터 세트 점수 : 0.00


### k값에 따른 정확도 구하기

In [142]:
tr_acc = []
test_acc = []
k_nums = range(1, 22, 2)# 1,3,5~21

for n in k_nums:
    # 모델 선택 및 학습 
    model_n = KNeighborsClassifier(n_neighbors=n)
    model_n.fit(X_train, y_train)
    
    # 정확도 구하기 
    acc_tr = model_n.score(X_train, y_train)
    acc_test = model_n.score(X_test, y_test)
    
    # 정확도 값 저장.
    tr_acc.append(acc_tr)
    test_acc.append(acc_test)
    
    print("k : ", n)
    print("학습용셋 정확도 {:.3f}".format(acc_tr) )
    print("테스트용셋 정확도 {:.3f}".format(acc_test) )

k :  1
학습용셋 정확도 0.977
테스트용셋 정확도 0.000
k :  3
학습용셋 정확도 0.325
테스트용셋 정확도 0.000
k :  5
학습용셋 정확도 0.213
테스트용셋 정확도 0.000
k :  7
학습용셋 정확도 0.146
테스트용셋 정확도 0.000
k :  9
학습용셋 정확도 0.102
테스트용셋 정확도 0.000
k :  11
학습용셋 정확도 0.079
테스트용셋 정확도 0.000
k :  13
학습용셋 정확도 0.067
테스트용셋 정확도 0.000
k :  15
학습용셋 정확도 0.056
테스트용셋 정확도 0.000
k :  17
학습용셋 정확도 0.051
테스트용셋 정확도 0.000
k :  19
학습용셋 정확도 0.049
테스트용셋 정확도 0.000
k :  21
학습용셋 정확도 0.046
테스트용셋 정확도 0.000


### 교차검증

In [143]:
from sklearn.model_selection import cross_val_score, KFold

kfold = KFold(n_splits=5, shuffle=True)
scores = cross_val_score(model, X_test, y_test, cv=kfold)
scores

array([0.        , 0.07692308, 0.07692308, 0.        , 0.        ])

In [144]:
import mglearn
from sklearn.linear_model import Ridge 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

### 릿지,랏쏘 모델 정확도

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
ridge = Ridge().fit(X_train, y_train)

print("훈련 데이터 세트 점수 : {:.2f}".format(lr.score(X_train, y_train)))
print("테스트 데이터 세트 점수 : {:.2f}".format(lr.score(X_test, y_test)))

훈련 데이터 세트 점수 : 0.03
테스트 데이터 세트 점수 : 0.00


In [146]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lasso = Lasso().fit(X_train, y_train)
print("학습용 데이터 세트 점수 : {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 데이터 세트 점수 : {:.2f}".format(lasso.score(X_test, y_test)))

학습용 데이터 세트 점수 : 0.03
테스트 데이터 세트 점수 : 0.00
